In [54]:
%matplotlib widget

import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import numpy.linalg as la


from skimage import io, transform
from skimage.util import img_as_ubyte
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, \
        BatchNormalization, Dense, Flatten, Activation, Input
from keras.utils.np_utils import to_categorical

# TeX typesetting
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)

# Aesthetics
plt.close('all')
plt.style.use('seaborn-pastel')

In [73]:
def bmatrix(a):
    """Returns a LaTeX bmatrix
    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

## (2)

#### (a)

In [55]:
image_size  = (222, 224)
batch_size  = 16
num_classes = 9
epochs = 20

In [56]:
## Building labels and training data
train_data = []
train_labels = []

classes = ["Coast", "Forest", "Highway", "Kitchen",\
                    "Mountain", "Office", "Store", "Street", "Suburb"]
for cl in classes:
    folder_path = os.path.join("../assets/assignment-6/dataset/train", cl)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        im = io.imread(fpath)
        im_resized = transform.resize(im, (224, 224) , anti_aliasing=True)
        im_arr = img_to_array(im_resized)
        train_data.append(im_arr)
        train_labels.append(classes.index(cl))
        
train_data = np.array(train_data)
train_labels = np.array(train_labels)
hot_enc_train_labels = to_categorical(train_labels, num_classes=num_classes)

#### (b)

In [57]:
## Model
## 3 iterations - convolute, batch norm, activation and max pooling
## flatten and output

model = Sequential([
    Conv2D(16, (2,2), input_shape=train_data.shape[1:]),
    BatchNormalization(),
    Activation("relu"),
    MaxPooling2D(2,2),
    
    Conv2D(32, (2,2)),
    BatchNormalization(),
    Activation("relu"),
    MaxPooling2D(2,2),
    
    Conv2D(64, (2,2)),
    BatchNormalization(),
    Activation("relu"),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(num_classes, activation="softmax")
])

In [58]:
model.compile(SGD(learning_rate=0.005), loss='categorical_crossentropy', metrics =['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_200 (Conv2D)          (None, 223, 223, 16)      80        
_________________________________________________________________
batch_normalization_200 (Bat (None, 223, 223, 16)      64        
_________________________________________________________________
activation_200 (Activation)  (None, 223, 223, 16)      0         
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_201 (Conv2D)          (None, 110, 110, 32)      2080      
_________________________________________________________________
batch_normalization_201 (Bat (None, 110, 110, 32)      128       
_________________________________________________________________
activation_201 (Activation)  (None, 110, 110, 32)     

In [59]:
history = model.fit(train_data, hot_enc_train_labels,
                    epochs=epochs,  
                    batch_size=batch_size,  
        ) 

Epoch 1/20
57/57 [==============================] - 15s 265ms/step - loss: 12.3239 - accuracy: 0.3044
Epoch 2/20
57/57 [==============================] - 18s 310ms/step - loss: 4.3037 - accuracy: 0.5289
Epoch 3/20
57/57 [==============================] - 17s 306ms/step - loss: 1.1798 - accuracy: 0.7656
Epoch 4/20
57/57 [==============================] - 17s 299ms/step - loss: 0.4661 - accuracy: 0.8711
Epoch 5/20
57/57 [==============================] - 17s 298ms/step - loss: 0.2131 - accuracy: 0.9244
Epoch 6/20
57/57 [==============================] - 17s 294ms/step - loss: 0.1938 - accuracy: 0.9311
Epoch 7/20
57/57 [==============================] - 17s 294ms/step - loss: 0.0770 - accuracy: 0.9767
Epoch 8/20
57/57 [==============================] - 17s 305ms/step - loss: 0.0349 - accuracy: 0.9878
Epoch 9/20
57/57 [==============================] - 18s 314ms/step - loss: 0.0255 - accuracy: 0.9933
Epoch 10/20
57/57 [==============================] - 21s 362ms/step - loss: 0.0173 - accur

In [104]:
test_data = []
test_labels = []
samples_per_class = [0] * num_classes

for i, cl in enumerate(classes):
    folder_path = os.path.join("../assets/assignment-6/dataset/test", cl)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        im = io.imread(fpath)
        im_resized = transform.resize(im, (224, 224) , anti_aliasing=True)
        im_arr = img_to_array(im_resized)
        test_data.append(im_arr)
        test_labels.append(classes.index(cl))
        samples_per_class[i] += 1
        
test_data = np.array(test_data)
test_labels = np.array(test_labels)
hot_enc_test_labels = to_categorical(test_labels, num_classes=num_classes)

In [61]:
model.evaluate(test_data, hot_enc_test_labels)

53/53 [==============================] - 11s 211ms/step - loss: 1.5894 - accuracy: 0.6625


[1.589385986328125, 0.6625368595123291]

#### (c)

In [63]:
data_aug = ImageDataGenerator(horizontal_flip=True, fill_mode="nearest")

In [64]:
## Model
## 3 iterations - convolute, batch norm, activation and max pooling
## flatten, dropout and output

sec_model = Sequential([
    Conv2D(16, (2,2), input_shape=train_data.shape[1:]),
    BatchNormalization(),
    Activation("relu"),
    MaxPooling2D(2,2),
    
    Conv2D(32, (2,2)),
    BatchNormalization(),
    Activation("relu"),
    MaxPooling2D(2,2),
    
    Conv2D(64, (2,2)),
    BatchNormalization(),
    Activation("relu"),
    MaxPooling2D(2,2),
    
    Dropout(0.5),
    Flatten(),
    Dense(num_classes, activation="softmax")
])

In [65]:
sec_model.compile(SGD(learning_rate=0.005), loss='categorical_crossentropy', metrics =['accuracy'])
sec_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_203 (Conv2D)          (None, 223, 223, 16)      80        
_________________________________________________________________
batch_normalization_203 (Bat (None, 223, 223, 16)      64        
_________________________________________________________________
activation_203 (Activation)  (None, 223, 223, 16)      0         
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_204 (Conv2D)          (None, 110, 110, 32)      2080      
_________________________________________________________________
batch_normalization_204 (Bat (None, 110, 110, 32)      128       
_________________________________________________________________
activation_204 (Activation)  (None, 110, 110, 32)     

In [66]:
sec_history = sec_model.fit(
    data_aug.flow(train_data,
                  hot_enc_train_labels,
                  batch_size=batch_size
    ),
    epochs=epochs,  
) 

Epoch 1/20
57/57 [==============================] - 14s 251ms/step - loss: 11.9324 - accuracy: 0.3378
Epoch 2/20
57/57 [==============================] - 17s 305ms/step - loss: 4.1172 - accuracy: 0.5278
Epoch 3/20
57/57 [==============================] - 18s 317ms/step - loss: 2.6938 - accuracy: 0.6033
Epoch 4/20
57/57 [==============================] - 18s 310ms/step - loss: 1.5345 - accuracy: 0.7133
Epoch 5/20
57/57 [==============================] - 17s 302ms/step - loss: 1.2084 - accuracy: 0.7378
Epoch 6/20
57/57 [==============================] - 18s 321ms/step - loss: 0.9366 - accuracy: 0.7978
Epoch 7/20
57/57 [==============================] - 18s 308ms/step - loss: 0.7311 - accuracy: 0.8244
Epoch 8/20
57/57 [==============================] - 18s 310ms/step - loss: 0.9985 - accuracy: 0.8267
Epoch 9/20
57/57 [==============================] - 17s 300ms/step - loss: 0.4445 - accuracy: 0.8833
Epoch 10/20
57/57 [==============================] - 18s 318ms/step - loss: 0.4795 - accur

In [67]:
sec_model.evaluate(test_data, hot_enc_test_labels)

53/53 [==============================] - 8s 157ms/step - loss: 1.3865 - accuracy: 0.7245


[1.3865114450454712, 0.7244837880134583]

#### (d)

In [146]:
def top_k_accuracy(model, data, labels, k = 5):
    top = 0.0
    probs = model.predict(data)
    for i in range(len(labels)):
        l = labels[i]
        prob = probs[i]
        top_values = (-prob).argsort()[:k] # descending order sort
        if np.isin(np.array([l]), top_values):
            top += 1.0
    return probs, top / len(labels)

In [147]:
probs, acc = top_k_accuracy(sec_model, test_data, test_labels, k=3)

In [148]:
"accuracy = {:.2f} %".format(100 * acc)

'accuracy = 93.63 %'

In [86]:
print(bmatrix(confusion_matrix(test_labels,  probs.argmax(axis=1))))

\begin{bmatrix}
  187 & 2 & 44 & 9 & 8 & 8 & 0 & 0 & 2\\
  4 & 187 & 0 & 4 & 15 & 3 & 10 & 5 & 0\\
  10 & 4 & 114 & 15 & 2 & 4 & 2 & 7 & 2\\
  0 & 0 & 0 & 82 & 0 & 16 & 8 & 1 & 3\\
  33 & 27 & 28 & 28 & 125 & 16 & 3 & 12 & 2\\
  0 & 0 & 0 & 12 & 0 & 103 & 0 & 0 & 0\\
  0 & 3 & 0 & 21 & 0 & 10 & 162 & 11 & 8\\
  0 & 2 & 2 & 10 & 1 & 7 & 12 & 153 & 5\\
  0 & 0 & 2 & 10 & 0 & 5 & 5 & 4 & 115\\
\end{bmatrix}


## (3)

#### (b)

In [48]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False, pooling="max", input_shape=(224,224,3))
base_model.trainable = False
                         
incept_model = Sequential([
    base_model,
    Dense(num_classes, activation="softmax")
])

incept_model.compile(optimizer=SGD(lr=0.005), loss='categorical_crossentropy', metrics=['accuracy'])
incept_model.summary()

preprocessed_train_data = preprocess_input(np.repeat(255. * train_data, 3, 3).astype(np.uint8))

incept_model.fit(preprocessed_train_data,
                 hot_enc_train_labels,
                 batch_size=batch_size,
                 epochs=epochs
) 

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 2048)              21802784  
_________________________________________________________________
dense_5 (Dense)              (None, 9)                 18441     
Total params: 21,821,225
Trainable params: 18,441
Non-trainable params: 21,802,784
_________________________________________________________________
Epoch 1/20
57/57 [==============================] - 47s 822ms/step - loss: 2.1661 - accuracy: 0.6089
Epoch 2/20
57/57 [==============================] - 46s 799ms/step - loss: 0.2943 - accuracy: 0.8989
Epoch 3/20
57/57 [==============================] - 47s 821ms/step - loss: 0.1582 - accuracy: 0.9478
Epoch 4/20
57/57 [==============================] - 47s 832ms/step - loss: 0.0831 - accuracy: 0.9733
Epoch 5/20
57/57 [==============================] - 40s 698ms/step - loss: 0.0523 - accuracy: 0

In [49]:
preprocessed_test_data = preprocess_input(np.repeat(255. * test_data,3,3).astype(np.uint8))
incept_model.evaluate(preprocessed_test_data, hot_enc_test_labels)

53/53 [==============================] - 77s 1s/step - loss: 0.1723 - accuracy: 0.9457


[0.1723114252090454, 0.9457226991653442]

In [50]:
incept_probs, incept_acc = top_k_accuracy(incept_model, preprocessed_test_data, test_labels, k=3)

In [51]:
"accuracy = {:.2f} %".format(100 * incept_acc)

'accuracy = 99.41 %'

In [143]:
matched = []
while len(matched) < 5:
    idx = np.random.randint(len(test_labels), size=1)[0]
    prob = incept_probs[idx]
    l = test_labels[idx]
    c = (-prob).argsort()[0]
    if  l != c:
        matched.append((idx+1, c))
        
print(matched)

for mat in matched:
    for i in range(9):
        if mat[0] < sum(samples_per_class[:i]):
            print(i-1, mat[0] - sum(samples_per_class[:i-1]))
            break;

[(627, 7), (462, 8), (816, 1), (235, 4), (105, 1)]
2 139
1 202
4 58
0 235
0 105
